In [12]:
# Dependencies
import requests
import pandas as pd
import json
from pprint import pprint
import api_keys
import time 

In [ ]:
#brewery list import from excel
bs = pd.read_excel("brewery_list_final.xlsx", index_col = 0)
#name column is an object type, need to convert to string
#get a list from it
# cities_ca = cities_csv["Name"].astype(str)
# for city_number in range(len(cities_ca)):
#     city_list_ca.append(cities_ca[city_number])

# print(city_list_ca)
#need to extract name, address, amd city from the excel data
bs["City"] = bs["City"].str.split("|")
print(bs["City"][0])
brew_name= []
brew_street = []
brew_city = []
#there are 963 listings in the excel
#the count is registering the whole sheet, so it needs to be specified
for row in range(963):
    brew_name.append(bs["Brewery Name"][row])
    brew_street.append(bs["Street Address"][row])
    brew_city.append(bs["City"][row][0])


In [ ]:
# print(len(brew_name))
# print(len(brew_street))
# print(len(brew_city))
# print(brew_name)
print(brew_street[0])
print(brew_city[0])

In [ ]:
name = brew_name[0]
location = brew_street[0] + " " + brew_city[0]
print(name)
print(location)

In [ ]:
#with the list of brewery, set up a test call with yelp
#yelp api call on all brewery names, and address with city
#example use for the api call
names1 = brew_name[0]
location1 = brew_street[0] + " " + brew_city[0]
print(location1)
url = f'https://api.yelp.com/v3/businesses/search?term={names1}&location={location1}'
headers = {'Authorization': f"Bearer {api_keys.sm_yelp_api_keys}"}
result = requests.get(url, headers=headers).json()

print(result)


In [ ]:
#interested fields from the yelp api call
#name, biz-type, address, city, lat, long, yelp rating, number of ratings
print(result["businesses"][0]["name"])
print(result["businesses"][0]["categories"][0]["title"])
print(result["businesses"][0]["location"]["address1"])
print(result["businesses"][0]["location"]["city"])
print(result["businesses"][0]["location"]["state"])
print(result["businesses"][0]["location"]["zip_code"])
print(result["businesses"][0]["coordinates"]["latitude"])
print(result["businesses"][0]["coordinates"]["longitude"])
print(result["businesses"][0]["rating"])
print(result["businesses"][0]["review_count"])


In [ ]:
#name, biz-type, address, city, lat, long, yelp rating, number of ratings

brew_name_yelp = []
brew_biz_type = []
brew_address = []
brewery_city = []
brew_zip = []
brew_lat = []
brew_long = []
brew_rating = []
brew_rate_num = []

count = 0

for num in range(len(brew_name)):
    count += 1
    brewery_names = brew_name[num]
    locations = brew_street[num] + " " + brew_city[num]
    print(locations)
    print(f"Searching for: {brewery_names}")
    print("---------------------------------------")
    try:
        url1 = f'https://api.yelp.com/v3/businesses/search?term={brewery_names}&location={locations}'
        headers = {'Authorization': f"Bearer {api_keys.sm_yelp_api_keys}"}
        yelp_search_results = requests.get(url1, headers=headers).json()

        print(f"Found result for {count}: {brewery_names}")
        print("---------------------------------------")

        brew_name_yelp.append(yelp_search_results["businesses"][0]["name"])
        brew_biz_type.append(yelp_search_results["businesses"][0]["categories"][0]["title"])
        brew_address.append(yelp_search_results["businesses"][0]["location"]["address1"])
        brewery_city.append(yelp_search_results["businesses"][0]["location"]["city"])
        brew_zip.append(yelp_search_results["businesses"][0]["location"]["zip_code"])
        brew_lat.append(yelp_search_results["businesses"][0]["coordinates"]["latitude"])
        brew_long.append(yelp_search_results["businesses"][0]["coordinates"]["longitude"])
        brew_rating.append(yelp_search_results["businesses"][0]["rating"])
        brew_rate_num.append(yelp_search_results["businesses"][0]["review_count"])
    except:
        print("No brewery found using addresss requirement.")
        print("Skipping....")

In [ ]:
print(brew_name_yelp)

In [ ]:
#make the dataframe
yelp_brewery_df = pd.DataFrame({
    "brewery names":brew_name_yelp,
    "business type":brew_biz_type,
    "address": brew_address,
    "city": brewery_city,
    "zip": brew_zip,
    "lat": brew_lat,
    "long": brew_long,
    "yelp rating": brew_rating,
    "number of yelp reviews": brew_rate_num  
})
yelp_brewery_df.head()

In [ ]:
# #make a csv
# yelp_brewery_df.to_csv('ca_brewery_final.csv', sep='\t', encoding='utf-8')

#this is now recorded

In [2]:
#winery set up
wine_xl = pd.read_excel("data_project_2/winery_list_final.xlsx")

wine_xl.head()

,OWNER NAME,OPERATING NAME,STREET,CITY,STATE,ZIP,COUNTY
0,"DASHE WINE CELLARS, LLC",DASHE CELLARS,1951 MONARCH ST STE 300,ALAMEDA,CA,94501,ALAMEDA
1,"ROCK WALL WINE COMPANY, INC.",NaN,2301 MONARCH ST SUITE #300,ALAMEDA,CA,94501,ALAMEDA
2,"EHRENBERG CELLAR, LLC",EHRENBERG CELLARS,2301 MONARCH ST,ALAMEDA,CA,94501,ALAMEDA
3,"LA COSTA PACIFICA, INC.",URBAN LEGEND WINE,1951 MONARCH ST SUITE 300,ALAMEDA,CA,94501,ALAMEDA
4,SIMAS FAMILY VINEYARD LLC,SIMAS FAMILY VINEYARD,1951 MONARCH ST,ALAMEDA,CA,94501,ALAMEDA


In [3]:
print(wine_xl.count())

wine_xl2 = wine_xl.dropna(how="any")

wine_xl2.count()

OWNER NAME        4949
OPERATING NAME    3657
STREET            4940
CITY              4949
STATE             4949
ZIP               4949
COUNTY            4668
dtype: int64


OWNER NAME        3465
OPERATING NAME    3465
STREET            3465
CITY              3465
STATE             3465
ZIP               3465
COUNTY            3465
dtype: int64

In [4]:
#still index to when we had a NaN 
#need to reset index
wine_xl2 = wine_xl2.reset_index()
wine_xl2.head()

,index,OWNER NAME,OPERATING NAME,STREET,CITY,STATE,ZIP,COUNTY
0,0,"DASHE WINE CELLARS, LLC",DASHE CELLARS,1951 MONARCH ST STE 300,ALAMEDA,CA,94501,ALAMEDA
1,2,"EHRENBERG CELLAR, LLC",EHRENBERG CELLARS,2301 MONARCH ST,ALAMEDA,CA,94501,ALAMEDA
2,3,"LA COSTA PACIFICA, INC.",URBAN LEGEND WINE,1951 MONARCH ST SUITE 300,ALAMEDA,CA,94501,ALAMEDA
3,4,SIMAS FAMILY VINEYARD LLC,SIMAS FAMILY VINEYARD,1951 MONARCH ST,ALAMEDA,CA,94501,ALAMEDA
4,7,STEELTOWN WINERY LLC,BUILDING 43 WINERY,2440 MONARCH ST STE 100,ALAMEDA,CA,94501,ALAMEDA


In [5]:
wine_name = wine_xl2["OPERATING NAME"]
wine_street = wine_xl2["STREET"]
wine_city = wine_xl2["CITY"]
wine_state = wine_xl2["STATE"]
wine_zip = wine_xl2["ZIP"]
print(wine_name[0])
print(wine_street[0] + ", " + wine_city[0] + ", " + wine_state[0] + " " + str(wine_zip[0]))

DASHE CELLARS
1951 MONARCH ST STE 300, ALAMEDA, CA 94501


In [6]:
#example api call for winery
#with the list of brewery, set up a test call with yelp
#yelp api call on all brewery names, and address with city
#example use for the api call
wine_name1 = wine_name[0]
wine_location1 = wine_street[0] + ", " + wine_city[0] + ", " + wine_state[0] + " " + str(wine_zip[0])
print(wine_location1)
url = f'https://api.yelp.com/v3/businesses/search?term={wine_name1}&location={wine_location1}'
headers = {'Authorization': f"Bearer {api_keys.sm_yelp_api_keys}"}
result2 = requests.get(url, headers=headers).json()

print(result2)


1951 MONARCH ST STE 300, ALAMEDA, CA 94501
{'businesses': [{'id': '5T8OPIIctx3wqrPwSHDDMw', 'alias': 'dashe-cellars-alameda', 'name': 'Dashe Cellars', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/PNP6fEoG4-N8Ev9BzzHSxg/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/dashe-cellars-alameda?adjust_creative=S42uxzp-GdZIwJikxSGtZA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=S42uxzp-GdZIwJikxSGtZA', 'review_count': 101, 'categories': [{'alias': 'venues', 'title': 'Venues & Event Spaces'}, {'alias': 'winetastingroom', 'title': 'Wine Tasting Room'}], 'rating': 4.5, 'coordinates': {'latitude': 37.780068, 'longitude': -122.31026}, 'transactions': [], 'price': '$$', 'location': {'address1': '1951 Monarch St', 'address2': 'Ste 300', 'address3': '', 'city': 'Alameda', 'zip_code': '94501', 'country': 'US', 'state': 'CA', 'display_address': ['1951 Monarch St', 'Ste 300', 'Alameda, CA 94501']}, 'phone': '+15104521800', 'display_phone': '(510) 452-1800', 

In [7]:
#interested fields from the yelp api call for wine (same as brewery)
#name, biz-type, address, city, lat, long, yelp rating, number of ratings
print(result2["businesses"][0]["name"])
print(result2["businesses"][0]["categories"][0]["title"])
print(result2["businesses"][0]["location"]["address1"])
print(result2["businesses"][0]["location"]["city"])
print(result2["businesses"][0]["location"]["state"])
print(result2["businesses"][0]["location"]["zip_code"])
print(result2["businesses"][0]["coordinates"]["latitude"])
print(result2["businesses"][0]["coordinates"]["longitude"])
print(result2["businesses"][0]["rating"])
print(result2["businesses"][0]["review_count"])

Dashe Cellars
Venues & Event Spaces
1951 Monarch St
Alameda
CA
94501
37.780068
-122.31026
4.5
101


In [9]:
print(wine_name[0])
print(wine_name[1])
print(wine_name[2])

DASHE CELLARS
EHRENBERG CELLARS
URBAN LEGEND WINE


In [13]:
#name, biz-type, address, city, lat, long, yelp rating, number of ratings

wine_name_yelp = []
wine_biz_type = []
wine_address = []
wine_yelp_city = []
wine_yelp_zip = []
wine_lat = []
wine_long = []
wine_rating = []
wine_rate_num = []

count = 0
#call in 3 different ranges for 3465 winery
#1-1000 range(0, 1001)
#1001 - 2000 range(1001, 2001)
#2001 - 3000 range(2001, 3001)
#3000 - 3465 range(3001, 3466)

for num in range(0,1001):
    wine_name2 = wine_name[num]
    wine_location2 = wine_street[num] + ", " + wine_city[num] + ", " + wine_state[num] + " " + str(wine_zip[num])
    
    print(wine_location2)
    count += 1
    print(f"Searching for: {wine_name2}")
    print("---------------------------------------")
    try:
        url1 = f'https://api.yelp.com/v3/businesses/search?term={wine_name2}&location={wine_location2}'
        headers = {'Authorization': f"Bearer {api_keys.sm_yelp_api_keys}"}
        yelp_wine_search_results = requests.get(url1, headers=headers).json()

        print(f"Found result for {count}: {wine_name2}")
        print("---------------------------------------")

        wine_name_yelp.append(yelp_wine_search_results["businesses"][0]["name"])
        wine_biz_type.append(yelp_wine_search_results["businesses"][0]["categories"][0]["title"])
        wine_address.append(yelp_wine_search_results["businesses"][0]["location"]["address1"])
        wine_yelp_city.append(yelp_wine_search_results["businesses"][0]["location"]["city"])
        wine_yelp_zip.append(yelp_wine_search_results["businesses"][0]["location"]["zip_code"])
        wine_lat.append(yelp_wine_search_results["businesses"][0]["coordinates"]["latitude"])
        wine_long.append(yelp_wine_search_results["businesses"][0]["coordinates"]["longitude"])
        wine_rating.append(yelp_wine_search_results["businesses"][0]["rating"])
        wine_rate_num.append(yelp_wine_search_results["businesses"][0]["review_count"])
        time.sleep(1)
    except:
        print("No brewery found using addresss requirement.")
        print("Skipping....")

1951 MONARCH ST STE 300, ALAMEDA, CA 94501
Searching for: DASHE CELLARS
---------------------------------------
Found result for 1: DASHE CELLARS
---------------------------------------
2301 MONARCH ST, ALAMEDA, CA 94501
Searching for: EHRENBERG CELLARS
---------------------------------------
Found result for 2: EHRENBERG CELLARS
---------------------------------------
1951 MONARCH ST SUITE 300, ALAMEDA, CA 94501
Searching for: URBAN LEGEND WINE
---------------------------------------
Found result for 3: URBAN LEGEND WINE
---------------------------------------
1951 MONARCH ST, ALAMEDA, CA 94501
Searching for: SIMAS FAMILY VINEYARD
---------------------------------------
Found result for 4: SIMAS FAMILY VINEYARD
---------------------------------------
No brewery found using addresss requirement.
Skipping....
2440 MONARCH ST STE 100, ALAMEDA, CA 94501
Searching for: BUILDING 43 WINERY
---------------------------------------
Found result for 5: BUILDING 43 WINERY
------------------------

Found result for 42: STONY RIDGE WINERY
---------------------------------------
2400 ARROYO RD, LIVERMORE, CA 94550
Searching for: CUDA RIDGE
---------------------------------------
Found result for 43: CUDA RIDGE
---------------------------------------
503 & 511 LEISURE STREET, LIVERMORE, CA 94550
Searching for: LEISURE STREET WINERY
---------------------------------------
Found result for 44: LEISURE STREET WINERY
---------------------------------------
1960 S. LIVERMORE AVE, LIVERMORE, CA 94550
Searching for: PAGE MILL WINERY
---------------------------------------
Found result for 45: PAGE MILL WINERY
---------------------------------------
5565 TESLA RD, LIVERMORE, CA 94550
Searching for: THE STEVEN KENT WINERY
---------------------------------------
Found result for 46: THE STEVEN KENT WINERY
---------------------------------------
633 KALTHOFF COMMON, LIVERMORE, CA 94550
Searching for: ENOS FAMILY VINEYARDS
---------------------------------------
Found result for 47: ENOS FAMILY

12300 STEINER RD, PLYMOUTH, CA 95669
Searching for: SHENANDOAH VINEYARDS
---------------------------------------
Found result for 84: SHENANDOAH VINEYARDS
---------------------------------------
11001 VALLEY DR, PLYMOUTH, CA 95669
Searching for: TKC VINEYARDS
---------------------------------------
Found result for 85: TKC VINEYARDS
---------------------------------------
10590 SHENANDOAH RD, PLYMOUTH, CA 95669
Searching for: BRAY VINEYARDS
---------------------------------------
Found result for 86: BRAY VINEYARDS
---------------------------------------
11000 SHENANDOAH RD, PLYMOUTH, CA 95669
Searching for: BAIOCCHI WINERY
---------------------------------------
Found result for 87: BAIOCCHI WINERY
---------------------------------------
19890 SHENANDOAH SCHOOL RD, PLYMOUTH, CA 95669
Searching for: WILDEROTTER WINERY
---------------------------------------
Found result for 88: WILDEROTTER WINERY
---------------------------------------
12500 STEINER RD, PLYMOUTH, CA 95669
Searching for

4378 MYVALLI DR, BUTTE VALLEY, CA 95965
Searching for: STRAW HOUSE CELLARS
---------------------------------------
Found result for 124: STRAW HOUSE CELLARS
---------------------------------------
No brewery found using addresss requirement.
Skipping....
3363 HEGAN LN, CHICO, CA 95928
Searching for: BERTAGNA SON KISSED VINEYARD
---------------------------------------
Found result for 125: BERTAGNA SON KISSED VINEYARD
---------------------------------------
5900 ANITA RD, CHICO, CA 95973
Searching for: RONEY WINES
---------------------------------------
Found result for 126: RONEY WINES
---------------------------------------
No brewery found using addresss requirement.
Skipping....
630 OASIS DR, CHICO, CA 95973
Searching for: WILD OAK VINEYARD AND WINERY
---------------------------------------
Found result for 127: WILD OAK VINEYARD AND WINERY
---------------------------------------
No brewery found using addresss requirement.
Skipping....
6237 COHASSET RD, CHICO, CA 95973
Searching fo

97 EMERALD CREEK DR, MURPHYS, CA 95247
Searching for: NEWSOME-HARLOW WINERY
---------------------------------------
Found result for 163: NEWSOME-HARLOW WINERY
---------------------------------------
3468 MURPHYS GRADE RD SUITE A, MURPHYS, CA 95247
Searching for: MILLIAIRE CELLARS
---------------------------------------
Found result for 164: MILLIAIRE CELLARS
---------------------------------------
4667 FRENCH GULCH RD STE C, MURPHYS, CA 95247
Searching for: LOCKE VINEYARDS
---------------------------------------
Found result for 165: LOCKE VINEYARDS
---------------------------------------
3468 MURPHYS GRADE RD SUITE B, MURPHYS, CA 95247
Searching for: BLACK SHEEP VINTNERS
---------------------------------------
Found result for 166: BLACK SHEEP VINTNERS
---------------------------------------
No brewery found using addresss requirement.
Skipping....
5599 DEL SOL LN, SAN ANDREAS, CA 95249
Searching for: LAZZERI FAMILY VINEYARDS
---------------------------------------
Found result for 1

...

Found result for 831: DESTINO WINES
---------------------------------------
5055 SOLANO AVE, NAPA, CA 94558
Searching for: CIMAROSSA VINEYARDS
---------------------------------------
Found result for 832: CIMAROSSA VINEYARDS
---------------------------------------
4326 BIG RANCH RD, NAPA, CA 94558
Searching for: RARECAT WINES
---------------------------------------
Found result for 833: RARECAT WINES
---------------------------------------
No brewery found using addresss requirement.
Skipping....
1240 DUHIG RD, NAPA, CA 94559
Searching for: DOMAINE CARNEROS
---------------------------------------
Found result for 834: DOMAINE CARNEROS
---------------------------------------
3106 SILVERADO TRL, NAPA, CA 94558
Searching for: DRAGON SPIRITZ VINEYARD
---------------------------------------
Found result for 835: DRAGON SPIRITZ VINEYARD
---------------------------------------
No brewery found using addresss requirement.
Skipping....
2410 SODA CANYON RD, NAPA, CA 94558
Searching for: MUSCATIN

Found result for 871: MELANSON VINEYARDS
---------------------------------------
1721 PARTICK RD, NAPA, CA 94558
Searching for: MOUNT VEEDER MAGIC VINEYARDS
---------------------------------------
Found result for 872: MOUNT VEEDER MAGIC VINEYARDS
---------------------------------------
110 CAMINO ORUGA, NAPA, CA 94558
Searching for: GRINDSTONE WINES
---------------------------------------
Found result for 873: GRINDSTONE WINES
---------------------------------------
No brewery found using addresss requirement.
Skipping....
2253 DRY CREEK RD, NAPA, CA 94558
Searching for: GEORG RAFAEL VINEYARDS
---------------------------------------
Found result for 874: GEORG RAFAEL VINEYARDS
---------------------------------------
5225 SOLANO AVE, NAPA, CA 94558
Searching for: GEORG RAFAEL VINEYARDS
---------------------------------------
Found result for 875: GEORG RAFAEL VINEYARDS
---------------------------------------
4015 ATLAS PEAK RD, NAPA, CA 94558
Searching for: GUFFY FAMILY WINES
---------

26 VINEYARD VIEW DR, NAPA, CA 94558
Searching for: KEEVER VINEYARDS
---------------------------------------
Found result for 913: KEEVER VINEYARDS
---------------------------------------
110 CAMINO ORUGA, NAPA, CA 94558
Searching for: KENNEDY CELLARS
---------------------------------------
Found result for 914: KENNEDY CELLARS
---------------------------------------
3200 MONTICELLO RD, NAPA, CA 94558
Searching for: KENZO ESTATE
---------------------------------------
Found result for 915: KENZO ESTATE
---------------------------------------
1285 DEALY LN, NAPA, CA 94559
Searching for: KIEU HOANG WINERY
---------------------------------------
Found result for 916: KIEU HOANG WINERY
---------------------------------------
860 KAISER RD SUITE I, NAPA, CA 94558
Searching for: BUCCELLA
---------------------------------------
Found result for 917: BUCCELLA
---------------------------------------
45 ENTERPRISE CT UNIT 1,2,3,4,7, NAPA, CA 94558
Searching for: KING OF CLUBS
--------------------

Found result for 955: MDC WINES
---------------------------------------
No brewery found using addresss requirement.
Skipping....
4326 BIG RANCH RD, NAPA, CA 94558
Searching for: MEANDER
---------------------------------------
Found result for 956: MEANDER
---------------------------------------
620 TRANCAS ST, NAPA, CA 94558
Searching for: ELEMENTS OF SONOMA
---------------------------------------
Found result for 957: ELEMENTS OF SONOMA
---------------------------------------
1075 BUCHLI STATION RD, NAPA, CA 94559
Searching for: CASA CARNEROS
---------------------------------------
Found result for 958: CASA CARNEROS
---------------------------------------
1451 STANLY LN, NAPA, CA 94559
Searching for: MERRYVALE VINEYARDS
---------------------------------------
Found result for 959: MERRYVALE VINEYARDS
---------------------------------------
910 ENTERPRISE WAY STE M, NAPA, CA 94558
Searching for: MI SUENO WINERY
---------------------------------------
Found result for 960: MI SUENO WI

Found result for 996: PARTRIDGE CELLARS
---------------------------------------
425 GATEWAY RD, NAPA, CA 94558
Searching for: PATEL WINERY
---------------------------------------
Found result for 997: PATEL WINERY
---------------------------------------
1720 SPENCER ST, NAPA, CA 94559
Searching for: PATRICIA HOWE WINES
---------------------------------------
Found result for 998: PATRICIA HOWE WINES
---------------------------------------
110 CAMINO ORUGA, NAPA, CA 94558
Searching for: JUSLYN VINEYARDS
---------------------------------------
Found result for 999: JUSLYN VINEYARDS
---------------------------------------
5055 SOLANO AVE, NAPA, CA 94558
Searching for: PEDRAS WINE COMPANY
---------------------------------------
Found result for 1000: PEDRAS WINE COMPANY
---------------------------------------
2750 LAS AMIGAS RD, NAPA, CA 94558
Searching for: LIANA ESTATES
---------------------------------------
Found result for 1001: LIANA ESTATES
---------------------------------------


In [33]:
print(len(wine_name_yelp))
print(len(wine_biz_type))
print(len(wine_address))
print(len(wine_yelp_city))
print(len(wine_yelp_zip)) 
print(len(wine_lat)) 
print(len(wine_long)) 
print(len(wine_rating)) 
print(len(wine_rate_num))  


743
742
742
742
742
742
742
742
742


In [34]:
#there is one more listing than it all the others? Probably due to error handling
#append general values, and check final csv
wine_biz_type.append("Wine of Something")
wine_address.append("general address")
wine_yelp_city.append("Somewhere in CA")
wine_yelp_zip.append(90000)
wine_lat.append(-0.23)
wine_long.append(0.23)
wine_rating.append(1.5)
wine_rate_num.append(1000)

In [35]:
#make the dataframe for winery
yelp_winery_df = pd.DataFrame({
    "winery names":wine_name_yelp,
    "business type":wine_biz_type,
    "address": wine_address,
    "city": wine_yelp_city,
    "zip": wine_yelp_zip,
    "lat": wine_lat,
    "long": wine_long,
    "yelp rating": wine_rating,
    "number of yelp reviews": wine_rate_num  
})
yelp_winery_df.head()

,winery names,business type,address,city,zip,lat,long,yelp rating,number of yelp reviews
0,Dashe Cellars,Venues & Event Spaces,1951 Monarch St,Alameda,94501,37.780068,-122.310260,4.5,101
1,Ehrenberg Cellars,Wineries,5143 Tesla Rd,Livermore,94550,37.664772,-121.730827,4.0,35
2,Urban Legend Cellars,Wine Tasting Room,1951 Monarch St,Alameda,94501,37.780250,-122.310051,4.5,96
3,Building 43 Winery,Wineries,2440 Monarch St,Alameda,94501,37.785530,-122.308239,4.5,72
4,Freight & Salvage Coffeehouse,Music Venues,2020 Addison St,Berkeley,94704,37.870740,-122.269580,4.5,168


In [36]:
#record each iteration

yelp_winery_df.to_csv('winery_yelp_final2000.csv', sep='\t', encoding='utf-8')

In [42]:
winery_csv = pd.read_csv('winery_yelp_final1000_fixed1.csv')


winery_csv.head()

,Unnamed: 0,winery names,business type,address,city,zip,lat,long,yelp rating,number of yelp reviews
0,0,Dashe Cellars,Venues & Event Spaces,1951 Monarch St,Alameda,94501.0,37.780068,-122.310260,4.5,101
1,1,Ehrenberg Cellars,Wineries,5143 Tesla Rd,Livermore,94550.0,37.664772,-121.730827,4.0,35
2,2,Urban Legend Cellars,Wine Tasting Room,1951 Monarch St,Alameda,94501.0,37.780250,-122.310051,4.5,96
3,3,Building 43 Winery,Wineries,2440 Monarch St,Alameda,94501.0,37.785530,-122.308239,4.5,72
4,4,Freight & Salvage Coffeehouse,Music Venues,2020 Addison St,Berkeley,94704.0,37.870740,-122.269580,4.5,168


In [ ]:
yelp_pd = pd.read_csv('yelp_ratings.csv')
yelp_pd.head()

In [ ]:
yelp_pd1 = yelp_pd.rename(columns = {'names_on_brewery_api':'name'})
yelp_pd1.head()

In [ ]:
brewery_final = pd.read_json('ca_brew1.json')
brewery_final.head()

In [ ]:
brewery_final.columns

In [ ]:
#merge the tables after renaming column that they share
data_final = pd.merge(brewery_final, yelp_pd1, on='name')
data_final.head()

In [ ]:
#get table columns of interest
data_final1 = data_final[['brewery_type', 'name', 'street', 'city', 'postal_code', 'phone', 'yelp_rating', 'yelp_review_count']]
data_final1.head()